In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import re
import csv
import pandas as pd
import re
from dateutil import parser
from datetime import date


import smtplib

In [15]:
# getting initial data pattern url, 10 pages loop

lista_links=[]
lista_nomes=[]
lista_nomes_links=[]


############################

# lista de ações a serem pesquisadas (fonte PLANILHA)
#https://docs.google.com/spreadsheets/d/1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY/edit#gid=610897588

nomes_acoes=['GOAU4','AGRO3','CMIN3','ABCB4','TAEE4','SAPR11','SAPR4','VULC3','ALOS3','TCSA3']
#nomes_acoes=['ABCB4']

# padrao de links para ações; constroi link completo a partir da lista
acoes_urls = [f"https://www.fundamentus.com.br/detalhes.php?papel={l}" for l in nomes_acoes]


############################


# laço

#for acao in acoes_urls:
for acao in nomes_acoes:
    #print(acao)
    #print(acoes_urls[0])
    
    URL = f"https://www.fundamentus.com.br/detalhes.php?papel={acao}"
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    souptext = BeautifulSoup(page.text)
    
    
    # pega todas as tabelas; a que precisa usar é a 3, logo usa-se a 2 abaixo
    tabela_indices=souptext.select('table.w728')
    tabela_certa=tabela_indices[2]
    #print(tabela_certa)
   
    
    ############################ 
    
    
    # separando informações relevantes (por linhas)
        #P/L
        #Margem Ebit % 
        #P/VP 
        #DY %
        #ROE %
        #Margem Liquida %
    
    
    # tabela para manipulação
    tabela_celula=[]
    
    # entrando nas linhas e colunas da tabela desejada
    for linha in tabela_certa:
        for coluna in linha:            
            
            # preparando dados e limpando
            # substituindo linha e % para usar
            result=re.sub(' +','', str(coluna))
            result=re.sub('\n','', str(coluna))
            result=re.sub('%','', str(result))
            #print(result)
            
            # separando linhas com dados, removendo dos labels etc
            result = re.search('<span class="txt">(.*)', str(result))
            #print(result)
            
            # apenas para manipulação, se vazia pula
            if(not result):
                pass
            # adiciona na tabela, caso estiver com alguma valor
            else:
                tabela_celula.append(result)
                #print(tabela_celula)
    
    
    
    ############################
    
    
    # Linhas para utilizar; abaixo
    #P/L 3,4
    #Margem Ebit % 20,21
    #P/VP 8,9
    #DY % 38,39
    #ROE % 40,41
    #Margem Liquida % 25,26
    
    
    # print para verificar dados com numeração
    #for i in range(0,len(tabela_celula)):
    #    print("\n")
    #    print(i, str(tabela_celula[i]))
    
        
   ############################


    # tabelas finais com as linhas
    tabela_final=[]
    tabela_indices=[]
    tabela_multiplos=[]
    
    
    #for i in [3,4,8,9,20,21,25,26,38,39,40,41]:
    for i in [3,4,20,21,8,9,38,39,40,41,25,26]:
        # substituindo com expressoes regulares, deixando apenas indices e multiplo
        tabela_celula[i]=re.sub(r'<re.Match object; span=(.*)\, match=\'<span class="txt">','', str(tabela_celula[i]))
        tabela_celula[i]=re.sub(r'</span></td>\'>','', str(tabela_celula[i]))

        #print(tabela_celula[i])
        
        # separando valores dos indices, apenas manipulação; 
        #se contem ',' é numero, se nao é string
        # se contem '-', recebe valor '0,0'
        # e se contem
        if(',' in tabela_celula[i]):
            tabela_multiplos.append(tabela_celula[i])
            
        elif('-' in tabela_celula[i]):
            tabela_multiplos.append('0,0')
            
        elif('-' not in tabela_celula[i]):
            tabela_indices.append(tabela_celula[i])
    
    
    ############################
    
    # esperando 1 segundo para enviar nova requisição
    time.sleep(1)
            
        
        
    ############################ inserindo data e ticker        
    # inserindo dia no inicio e convertendo formato
    hoje = date.today()
    new_hoje = hoje.strftime("%d/%m/%Y")
    #print(new_hoje)
    
    # isnerindo ticker realizado
    tabela_multiplos.insert(0, acao)        
    tabela_indices.insert(0, "Acao")   

    # isnerindo dia realizado
    tabela_multiplos.insert(1, new_hoje)        
    tabela_indices.insert(1, "Data coleta")   
    
    
    ############################

    
    # print verficação
    #print(len(tabela_indices))
    #print(tabela_indices)
    #print(len(tabela_multiplos))
    #print(tabela_multiplos)
    
    ############################
    
    ##### não mais necessário
    #agregating listas em um dataframe
    #acoes_scraping = pd.DataFrame(
    #    {'Acao': nomes_acoes,
    #     'Multiplos': tabela_multiplos
    #    })


    ############################ criando e salvando lista em dataframe
    acoes_scraping = pd.DataFrame(tabela_multiplos)


    # transposing colunas e linhas, para visualização

    acoes_scraping=acoes_scraping.T

    acoes_scraping.to_csv("acoes_scraping.csv", mode='a', header=False,index=False)
    print(f'@@@@@@@@@@ Linha escrita no arquivo, {acao} @@@@@@@@@@')
    
print('\n\n ############################ Ordem dos indicadores ############################')
print('\n', tabela_indices)
print(nomes_acoes)

@@@@@@@@@@ Linha escrita no arquivo, GOAU4 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, AGRO3 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, CMIN3 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, ABCB4 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, TAEE4 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, SAPR11 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, SAPR4 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, VULC3 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, ALOS3 @@@@@@@@@@
@@@@@@@@@@ Linha escrita no arquivo, TCSA3 @@@@@@@@@@


 ############################ Ordem dos indicadores ############################

 ['Acao', 'Data coleta', 'P/L', 'Marg. EBIT', 'P/VP', 'Div. Yield', 'ROE', 'Marg. Líquida']
['GOAU4', 'AGRO3', 'CMIN3', 'ABCB4', 'TAEE4', 'SAPR11', 'SAPR4', 'VULC3', 'ALOS3', 'TCSA3']
